In [1]:
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import time
import csv
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import threading

In [2]:
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #認証情報設定
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    #アクセス
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).get_worksheet(0)
    
    return worksheet

# jsonfile
jsonf = "ikkyosai-order-046c29b43754.json"
# スプレットシートキー指定
spread_sheet_key = "1UrCkBZWG0SGWIhxbhBUYVIoxOA_x49UqQ-6oGV3JRmg"
ws = connect_gspread(jsonf,spread_sheet_key)


In [3]:
def connect_gspread2(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #認証情報設定
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    #アクセス
    SPREADSHEET_KEY = key
    worksheet2 = gc.open_by_key(SPREADSHEET_KEY).get_worksheet(1)
    
    return worksheet2

# jsonfile
jsonf = "ikkyosai-order-046c29b43754.json"
# スプレットシートキー指定
spread_sheet_key = "1UrCkBZWG0SGWIhxbhBUYVIoxOA_x49UqQ-6oGV3JRmg"
ws2 = connect_gspread2(jsonf,spread_sheet_key)

def reload2():
  # jsonfile
  jsonf = "ikkyosai-order-046c29b43754.json"
  # スプレットシートキー指定
  spread_sheet_key = "1UrCkBZWG0SGWIhxbhBUYVIoxOA_x49UqQ-6oGV3JRmg"
  ws2 = connect_gspread2(jsonf,spread_sheet_key)
  with open('output2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(ws2.get_all_values())
  df2=pd.read_csv("output2.csv",encoding="shift-jis")
reload2()
df2=pd.read_csv("output2.csv",encoding="shift-jis")

In [4]:
def reload():
  # jsonfile
  jsonf = "ikkyosai-order-046c29b43754.json"
  # スプレットシートキー指定
  spread_sheet_key = "1UrCkBZWG0SGWIhxbhBUYVIoxOA_x49UqQ-6oGV3JRmg"
  ws = connect_gspread(jsonf,spread_sheet_key)
  with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(ws.get_all_values())
  df=pd.read_csv("output.csv",encoding="shift-jis")
reload()

In [5]:
def money_count():
  with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(ws.get_all_values())
  df=pd.read_csv("output.csv",encoding="shift-jis")
  df=df.fillna("a")
  for i in range(len(df)):
    if df.iloc[i,4]=="a" or df.iloc[i,4]==0:
      time.sleep(1)
      pay=0
      order=df.iloc[i,2]
      order_list=order.split(", ")
      for divide_order in order_list:
        if divide_order=="飲み物のみ":
            pay=200
        elif "セット" in divide_order:
            pay+=500
        else:
            pay+=400
      ws.update_cell(i+2,5,pay)
  def show_reload_done():
    popup = tk.Toplevel()
    popup.title("更新完了")
    
    # ラベルの作成
    label = ttk.Label(popup, text="更新が完了しました。", background='#7fffd4', foreground='#333333')
    label.pack(padx=20, pady=20)
    
    # OKボタンの作成
    ok_button = ttk.Button(popup, text="OK", command=popup.destroy)
    ok_button.pack(pady=10)
    
    # ポップアップの表示
    popup.mainloop()
  reload()
  show_reload_done()

In [ ]:
#配膳実施システム（配膳済入力用）　
def haizen(number):
    with open('output.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerows(ws.get_all_values())
    df=pd.read_csv("output.csv",encoding="shift-jis")
    df=df.fillna("a")
    flag=0
    numbers=["1","2","3","4","5","6","7","8","9","0"]
    def kakunin_popup():
      with open('output.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(ws.get_all_values())
      df=pd.read_csv("output.csv",encoding="shift-jis")
      a=0
      popup = tk.Toplevel()
      popup.title("確認")
      popup.geometry("600x600")
      popup.configure(bg='#7fffd4')
      label = ttk.Label(popup, text="座席番号"+str(number)+
                        "番、ご注文内容は"+df.iloc[i,2]+df.iloc[i,3]+"でよろしいでしょうか",
                          background='#7fffd4', foreground='#333333')
      label.pack(pady=20)
      button_frame = ttk.Frame(popup)
      button_frame.pack(pady=20)
      def haizen_yes():
        ws.update_cell(i+2,6,"配膳済")
        popup.destroy()
      yes_button = ttk.Button(button_frame, text="はい", command=haizen_yes, style='MyButton.TButton', width=10)
      yes_button.pack(side="left", padx=10)
      yes_button = ttk.Button(button_frame, text="いいえ", command=popup.destroy, style='MyButton.TButton', width=10)
      yes_button.pack(side="right", padx=10)
      popup.wait_window()
      return(a)
    for i in number:
      if i not in numbers:
         flag=1
    if flag==0:
       number=int(number)
       for i in range(len(df)):
         if df.iloc[i,1]==number and not df.iloc[i,6] =="会計済" and not df.iloc[i,5]=="配膳済"and not df.iloc[i,7]=="注文ミス":
            kakunin_popup()
            

    else:
      root = tk.Tk()
      root.withdraw()
      messagebox.showerror("エラー", "半角数字で入力してください")

In [ ]:
#会計実施システム（会計済入力用）　
def kaikei(number):
    with open('output.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerows(ws.get_all_values())
    df=pd.read_csv("output.csv",encoding="shift-jis")
    df=df.fillna("a")
    flag=0
    numbers=["1","2","3","4","5","6","7","8","9","0"]
    def kakunin_popup():
      with open('output.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(ws.get_all_values())
      df=pd.read_csv("output.csv",encoding="shift-jis")
      a=0
      popup = tk.Toplevel()
      popup.title("確認")
      popup.geometry("600x600")
      popup.configure(bg='#7fffd4')
      label = ttk.Label(popup, text="座席番号"+str(number)+
                        "番、ご注文内容は"+df.iloc[i,2]+df.iloc[i,3]+
                        ",お会計は"+str(df.iloc[i,4])+"円で間違えないですか？",
                          background='#7fffd4', foreground='#333333')
      label.pack(pady=20)
      button_frame = ttk.Frame(popup)
      button_frame.pack(pady=20)
      def yes():
        ws.update_cell(i+2,7,"会計済")
        popup.destroy()
      yes_button = ttk.Button(button_frame, text="はい", command=yes, style='MyButton.TButton', width=10)
      yes_button.pack(side="left", padx=10)
      yes_button = ttk.Button(button_frame, text="いいえ", command=popup.destroy, style='MyButton.TButton', width=10)
      yes_button.pack(side="right", padx=10)
      popup.wait_window()
      return(a)
    for i in number:
      if i not in numbers:
         flag=1
    if flag==0:
       number=int(number)
       for i in range(len(df)):
         if df.iloc[i,1]==number and not df.iloc[i,6] =="会計済" and not df.iloc[i,7]=="注文ミス":
            kakunin_popup()
            

    else:
      root = tk.Tk()
      root.withdraw()
      messagebox.showerror("エラー", "半角数字で入力してください")

In [8]:
def show_kaikei_popup():
    with open('output.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerows(ws.get_all_values())
    df=pd.read_csv("output.csv",encoding="shift-jis")
    
    popup = tk.Toplevel()
    popup.title("入力ポップアップ")
    
    # ポップアップの大きさを変更
    popup.geometry("400x200")
    
    # ポップアップの背景色を変更
    popup.configure(bg='#7fffd4')
    
    # テキストボックスを作成
    entry = ttk.Entry(popup, width=30)
    entry.pack(pady=20)
    
    # ボタンのスタイルを変更
    style = ttk.Style()
    style.configure('MyButton.TButton', background='#7fffd4', font=('Arial', 14), padding=10)
    
    # ボタンの配置を変更
    button_frame = ttk.Frame(popup)
    button_frame.pack(pady=20)
    
    def get_text():
        text = entry.get()
        kaikei(text)
        popup.destroy()
    
    submit_button = ttk.Button(button_frame, text="送信", command=get_text, style='MyButton.TButton', width=10)
    submit_button.pack(side="right", padx=10)
    
    popup.mainloop()

In [9]:
def show_haizen_popup():
    with open('output.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerows(ws.get_all_values())
    df=pd.read_csv("output.csv",encoding="shift-jis")
    
    popup = tk.Toplevel()
    popup.title("入力ポップアップ")
    
    # ポップアップの大きさを変更
    popup.geometry("400x200")
    
    # ポップアップの背景色を変更
    popup.configure(bg='#7fffd4')
    
    # テキストボックスを作成
    entry = ttk.Entry(popup, width=30)
    entry.pack(pady=20)
    
    # ボタンのスタイルを変更
    style = ttk.Style()
    style.configure('MyButton.TButton', background='#7fffd4', font=('Arial', 14), padding=10)
    
    # ボタンの配置を変更
    button_frame = ttk.Frame(popup)
    button_frame.pack(pady=20)
    
    def get_text():
        text = entry.get()
        haizen(text)
        popup.destroy()
    
    submit_button = ttk.Button(button_frame, text="送信", command=get_text, style='MyButton.TButton', width=10)
    submit_button.pack(side="right", padx=10)
    
    popup.mainloop()

In [10]:
def show_popup():
    popup = tk.Toplevel()
    popup.title("ハルモニア会計システム")
    
    # ポップアップの大きさを変更
    popup.geometry("600x600")
    
    # ポップアップの背景色を変更
    popup.configure(bg='#7fffd4')
    
    # ポップアップの内容を設定
    label = ttk.Label(popup, text="ハルモニア会計システム", background='#7fffd4', foreground='#333333')
    label.pack(pady=20)

    button_frame = ttk.Frame(popup)
    button_frame.pack(pady=50)
    
    # ボタンのスタイルを変更
    style = ttk.Style()
    style.configure('MyButton.TButton', background='#7fffd4', font=('Arial', 14), padding=10)
    
    # ボタンの大きさを変更
    kaikei_button = ttk.Button(button_frame, text="配膳", command=show_haizen_popup, style='MyButton.TButton', width=20)
    kaikei_button.pack(pady=20)
    kaikei_button = ttk.Button(button_frame, text="会計", command=show_kaikei_popup, style='MyButton.TButton', width=20)
    kaikei_button.pack(pady=20)
    count_button = ttk.Button(button_frame, text="会計更新", command=money_count, style='MyButton.TButton', width=20)
    count_button.pack(pady=20)
    popup.mainloop()

In [11]:
#座席に注文を入れる際の引数の作成
#values = [df.iloc[x, y] for x, y in positions]のように書くこと
#変更後
order_list_list=[]
df2=df2.fillna("cat!?")
for i in range(40):
    order_list_list.append([])
count=0
order_list_for_count=[2,4,7,9,12,14,17,19]
for i in range(8):
  for s in range(4):
    order_list_list[i+count].append((4+s,order_list_for_count[i]))
  count+=1
count=0
for i in range(8):
  for s in range(4):
    order_list_list[i+count+1].append((9+s,order_list_for_count[i]))
  count+=1
order_count_2=[2,4,7,9,12,14,17,19]
count=0
for i in range(8):
  for s in range(4):
    order_list_list[i+count+16].append((15+s,order_count_2[i]))
  count+=1
count=0
for i in range(8):
  for s in range(4):
    order_list_list[i+count+17].append((20+s,order_count_2[i]))
  count+=1

In [12]:
#座席に紐づけて注文状況を入れる際の引数の作成
#values = [df.iloc[x, y] for x, y in positions]のように書くこと
#変更後
order_check_list_list=[]
df2=df2.fillna("cat!?")
for i in range(40):
    order_check_list_list.append([])
count=0
order_list_for_count=[2,4,7,9,12,14,17,19]
for i in range(8):
  for s in range(4):
    order_check_list_list[i+count].append((4+s,order_list_for_count[i]+1))
  count+=1
count=0
for i in range(8):
  for s in range(4):
    order_check_list_list[i+count+1].append((9+s,order_list_for_count[i]+1))
  count+=1
order_count_2=[2,4,7,9,12,13,17,19]
count=0
for i in range(8):
  for s in range(4):
    order_check_list_list[i+count+16].append((15+s,order_count_2[i]+1))
  count+=1
count=0
for i in range(8):
  for s in range(4):
    order_check_list_list[i+count+17].append((20+s,order_count_2[i]+1))
  count+=1
print(order_check_list_list[33])

[]


In [13]:
def reset2():
  with open("output2.csv", mode='r', newline='', encoding='shift_jis') as infile2:
    reader2 = csv.reader(infile2)
    rows_2 = [row for row in reader2]

  for i in order_check_list_list:
    for x,y in i:
        rows_2[x][y]=""
  for i in order_list_list:
    for x,y in i:
        rows_2[x][y]==""
  with open("output2.csv", mode='w', newline='', encoding='shift_jis') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(rows_2)
reset2()

In [14]:
orderater={"【セット】チーズケーキ＋飲み物":"チーズ","【セット】抹茶スコーン＋飲み物":"抹茶","【セット】チョコブラウニー＋飲み物":"チョコ","【セット】バナナのパウンドケーキ＋飲み物":"バナナ","【セット】ほうれん草とベーコンのキッシュ＋飲み物":"キシュ","【単品】チーズケーキ":"チーズ単","【単品】抹茶スコーン":"抹茶単","【単品】チョコブラウニー":"チョコ単","【単品】バナナのパウンドケーキ":"バナナ単","【単品】ほうれん草とベーコンのキッシュ":"キシュ単",
           "リンゴジュース":"林檎","アイスティー":"冷茶","ホットティー":"温茶","アイスコーヒー":"冷コ","ホットコーヒー":"温コ","オレンジジュース":"蜜柑","ケーキ単品を注文しました":"単","飲み物のみ":""}

In [15]:
def timekeeper(a):
  a_list=a.split(" ")
  a_list=a_list[1].split(":")
  return a_list

In [16]:
from datetime import datetime
def timer(late):
  now=str(datetime.now())
  now=timekeeper(now)
  now=int(now[0])*60+int(now[1])
  late=timekeeper(late)
  late=int(late[0])*60+int(late[1])
  if now-late>30:
    return "over"
  else:
    return "ok"
timer("2024/09/20 10:59:33")

'ok'

In [17]:
def seating():
  a=0
  while not a=="finish":
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    order=""
    #リセット処理
    with open("reset.csv", mode='r', newline='', encoding='shift_jis') as resetfile:
      reader_reset = csv.reader(resetfile)
      rows_reset = [row for row in reader_reset]
    with open("output2.csv", mode='w', newline='', encoding='shift_jis') as outfile:
      writer = csv.writer(outfile)
      writer.writerows(rows_reset)
    #リセット処理終わり
    reload()
    with open("output.csv", mode='r', newline='', encoding='shift_jis') as infile:
      reader = csv.reader(infile)
      rows_1 = [row for row in reader]
      a=rows_1[0][8]
    with open("output2.csv", mode='r', newline='', encoding='shift_jis') as infile2:
      reader2 = csv.reader(infile2)
      rows_2 = [row for row in reader2]
    for i in range(len(rows_1)):
      if not rows_1[i][7] in ["注文ミス","１日目","２日目"]:
        if "チーズ" in rows_1[i][2]:
           count1+=1
        elif "抹茶" in rows_1[i][2]:
           count2+=1
        elif "チョコ" in rows_1[i][2]:
           count3+=1
        elif "バナナ" in rows_1[i][2]:
         count4+=1
        elif "キッシュ" in rows_1[i][2]:
         count5+=1
        if not rows_1[i][6]=="会計済":
           order=orderater[rows_1[i][2]]+orderater[rows_1[i][3]]
           for x,y in order_list_list[int(rows_1[i][1])-1]:
            if rows_2[x][y]=="":
              rows_2[x][y]=order
              if timer(rows_1[i][0])=="over":
                rows_2[x][y+1]="過"
              elif rows_1[i][5]=="配膳済":
                rows_2[x][y+1]="済"
              break
    rows_2[15][25]=int(rows_reset[15][25])-count1
    rows_2[16][25]=int(rows_reset[16][25])-count2
    rows_2[17][25]=int(rows_reset[17][25])-count3
    rows_2[18][25]=int(rows_reset[18][25])-count4
    rows_2[19][25]=int(rows_reset[19][25])-count5
    with open("output2.csv", mode='w', newline='', encoding='shift_jis') as outfile:
      writer = csv.writer(outfile)
      writer.writerows(rows_2)
    ws.update(range_name="A1",values=rows_1)
    ws2.update(range_name="A1",values=rows_2)
    time.sleep(10)

      




In [18]:
def start():
  # スレッドを作成
  thread1 = threading.Thread(target=seating)
  thread2 = threading.Thread(target=show_popup)

  # スレッドを開始
  thread1.start()
  thread2.start()

# スレッドの終了を待機
  thread1.join()
  thread2.join()

In [ ]:
start()